In [ ]:
pip install exchange_calendars

# **Selección de Activos**

**Selección de Activos**


1.   **AGG**: ETF que da exposición a bonos de grado de inversión en EE.UU. Toma como indice de referencia el Bloomber US Aggregate Bonds Index. Con principal enfoque en bonos emitidos por el tesoro estadounidense, el sector industrial y bonos de hipotecas respaldadas por activos. Donde los bonos que lo conforman cuentan con calidad crediticia desde AAA hasta BBB, donde el 70% del portafolio se encuentra conformado por bonos AA. Donde nuestros activos cuentan con vencimiento desde 1 hasta más de 20 años. Mayormente el etf cuenta con bonos de 7 a 10 años. Con una comisión del 0.03%. Con una Beta de 0.24 y un Yield de 3.19%. Con convexidad de 0.64. Con una desviación estandar de 6.25%. Divisa base en USD.
2.   **SEMB.L**: ETF que da exposición a bonos de mercados emergentes emitidos en dolares, con principal peso en bonos soberanos y bonos de propiedad sin garantia, mayormente en los siguientes paises: México, Arabia Saudita, Turquia, indonesia, Emiratos Arabes, Qatar, China y Brasil. Con vencimiento de los instrumentos a plazos mayores a 20 años, de 3 a 5 y de 7 a 10 años. Con una comisión por gestion del 0.45%. Con  divisa base en Dolares Estadounidenses. Desviación Tipica de 10.80%. Duración efectiva de 6.95. Beta de 1. Que toma como indice de referencia al JP Morgan EMBI Global Core Index.
3.   **IYW**:ETF de empresas de software, hardware y tecnologias de la información en EEUU. Divisa base en USD. Beta de 1.19, Desviacion Estandar de 23.62%. Comision de administracion de 0.40%. Con principales posiciones en: APPLE INC, MICROSOFT CORP, ALPHABET INC CLASS A, NVIDIA CORP, META PLATFORMS INC CLASS A. Toma como indice de referencia al Rusell 1000 Technology RIC 22.5/45 Capped Index.
4.   **IEMS.L**: ETF que da exposición diversificada a mercados emergentes: India, Taiwan, Corea, China, Brasil, México, entre otros.Con principal enfoque en tecnologia e información, industria, materiales y consumo discrecional. Divisa base en USD. Comision por gestion de 0.74%. Beta de 0.998. Desviación de 17.21%. Define como Benchmark el MSCI Emergentic Markets Small Cap Index.
5.   **GSG**: ETF de exposicón a materias primas con principal enfoque en ganaderia, agricultura, energia, metales industriales y preciosos. Con una beta de 0.40 y una desviación estandar de 20.51%. Toma como indice de referencia el S&P GSCI Total Return. Divisa Base en USD

# **Paqueteria**

In [ ]:
# Importa librerias
from numpy import *
import numpy as np
from numpy.linalg import multi_dot
import pandas as pd
import yfinance as yf
import plotly as plt
from datetime import date, datetime
import exchange_calendars as xcals
from scipy.stats import norm, skew, kurtosis
import cufflinks as cf
cf.set_config_file(offline=True)
import scipy.optimize as sco
from scipy.optimize import minimize


# Libreria especifica para ignorar warnings
import warnings
warnings.filterwarnings('ignore')

# **Estadisticas de activos**: Obtención de rendimientos

Definimos los tickers que vamos a utilizar

In [ ]:
# Tickers
symbols = ['AGG', 'SEMB.L', 'IYW', 'IEMS.L', 'GSG', 'MXN=X', 'GBPMXN=X' ]
etfs = ['AGG', 'SEMB.L', 'IYW', 'IEMS.L', 'GSG']

# Number of assets
numofasset = len(symbols)
numofasset2 = numofasset-2

# Número de portafolios a optimizar
numofportfolio = 1000

Funcion para bajar datos de Yahoo Finance

In [ ]:
def download_data(tickers, start_date='2010-01-01', end_date=date.today().strftime('%Y-%m-%d')):
    data = yf.download(tickers, start=start_date, end=end_date)

    return data['Close']

Obtencion de los datos de nuestros tickers

In [ ]:
df=download_data(symbols)
df = df.dropna()
df

[*********************100%%**********************]  7 of 7 completed


,AGG,GBPMXN=X,GSG,IEMS.L,IYW,MXN=X,SEMB.L
Date,,,,,,,
2010-01-04,103.309998,20.739000,32.700001,63.380001,14.622500,12.871800,6312.120117
2010-01-05,103.779999,20.472000,33.029999,63.799999,14.600000,12.810800,6365.080078
2010-01-06,103.720001,20.410999,33.270000,64.589996,14.497500,12.739500,6361.410156
2010-01-07,103.599998,20.329000,33.110001,64.309998,14.442500,12.760200,6389.250000
2010-01-08,103.660004,20.322001,33.180000,64.620003,14.552500,12.684200,6363.479980
...,...,...,...,...,...,...,...
2023-12-04,96.459999,21.849300,20.330000,83.105003,116.190002,17.188761,6706.500000
2023-12-05,97.099998,22.064600,20.180000,82.839996,116.820000,17.454100,6773.500000
2023-12-06,97.349998,21.861425,19.620001,83.309998,115.720001,17.358299,6811.000000


In [ ]:
#Pasando a pesos todo
df[etfs[0]] = df[etfs[0]] * df["MXN=X"]
df[etfs[1]] = df[etfs[1]] * df["GBPMXN=X"]
df[etfs[2]] = df[etfs[2]]* df["MXN=X"]
df[etfs[3]] = df[etfs[3]] * df["MXN=X"]
df[etfs[4]] = df[etfs[4]] * df["MXN=X"]

Obtenemos Rendimientos de los activos

In [ ]:
#Calculando Retornos
returns = df.pct_change().fillna(0)
tasasdecambio = ["GBPMXN=X", "MXN=X"]
returns = returns.drop(tasasdecambio, axis = 1)
returns_1 = returns.copy()

Vamos a definir los datos para los dias del mercado mexicano.

In [ ]:
# Obtén el calendario para la Bolsa de Mexicana
mex_cal = xcals.get_calendar('XMEX')

Vamos a definir una funcion para obtener las distintas ventanas de tiempo.


In [ ]:
def calcular_fechas(hoy: pd.Timestamp):
    # Obtén el calendario de la bolsa de México
    xmex = xcals.get_calendar("XMEX")

    # Si el día de la semana es lunes (0 en el sistema Python weekday()), retrocede 3 días
    if hoy.weekday() == 0:
        prev_business_day = hoy - pd.Timedelta(days=3)
    # De lo contrario, solo retrocede un día
    else:
        prev_business_day = hoy - pd.Timedelta(days=1)

    # Si el día calculado no es un día hábil, busca el día hábil más reciente
    if not xmex.is_session(prev_business_day):
        prev_business_day = xmex.previous_close(prev_business_day).to_pydatetime()

    ayer = prev_business_day

    # Crear un diccionario para almacenar los resultados
    resultado = {}

    # Mes hasta la fecha
    primer_dia_mes = xmex.date_to_session(hoy.replace(day=1), direction="next")
    if hoy == primer_dia_mes:
        # Si hoy es el primer día hábil del mes, toma el primer día hábil del mes anterior
        mes_anterior = hoy - pd.DateOffset(months=1)
        primer_dia_mes = xmex.date_to_session(mes_anterior.replace(day=1), direction="next")

    # Calcula los días hábiles entre el primer día del mes y hoy
    dias_habiles = len(xmex.sessions_in_range(primer_dia_mes, hoy))+1

    # Usa estos días hábiles para obtener la ventana de sesiones
    mes_hasta_fecha = xmex.sessions_window(hoy, -dias_habiles)

    # Año hasta la fecha
    primer_dia_año = xmex.date_to_session(hoy.replace(month=1, day=1), direction="next")
    if hoy == primer_dia_año:
        # Si hoy es el primer día hábil del año, toma el primer día hábil del año anterior
        año_anterior = hoy - pd.DateOffset(years=1)
        primer_dia_año = xmex.date_to_session(año_anterior.replace(month=1, day=1), direction="next")

    # Calcula los días hábiles entre el primer día del año y hoy
    dias_habiles = len(xmex.sessions_in_range(primer_dia_año, hoy))+1

    # Usa estos días hábiles para obtener la ventana de sesiones
    año_hasta_fecha = xmex.sessions_window(hoy, -dias_habiles)

    # Fecha de hace un mes
    hace_un_mes = hoy - pd.DateOffset(months=1)

    # Encuentra el día hábil más cercano en el pasado a hace_un_mes
    dia_habil_hace_un_mes = xmex.date_to_session(hace_un_mes, direction="previous")

    # Obtén todas las sesiones desde hace_un_mes hasta hoy
    ultimos_30_dias = xmex.sessions_in_range(dia_habil_hace_un_mes, hoy)

    # Fecha de hace tres meses
    hace_tres_meses = hoy - pd.DateOffset(months=3)

    # Encuentra el día hábil más cercano en el pasado a hace_tres_meses
    dia_habil_hace_tres_meses = xmex.date_to_session(hace_tres_meses, direction="previous")

    # Obtén todas las sesiones desde hace_tres_meses hasta hoy
    ultimos_90_dias = xmex.sessions_in_range(dia_habil_hace_tres_meses, hoy)

    # Fecha de hace seis meses
    hace_seis_meses = hoy - pd.DateOffset(months=6)

    # Encuentra el día hábil más cercano en el pasado a hace_seis_meses
    dia_habil_hace_seis_meses = xmex.date_to_session(hace_seis_meses, direction="previous")

    # Obtén todas las sesiones desde hace_seis_meses hasta hoy
    ultimos_180_dias = xmex.sessions_in_range(dia_habil_hace_seis_meses, hoy)

    # Fecha de hace un año
    hace_un_año = hoy - pd.DateOffset(years=1)

    # Encuentra el día hábil más cercano en el pasado a hace_un_año
    dia_habil_hace_un_año = xmex.date_to_session(hace_un_año, direction="previous")

    # Obtén todas las sesiones desde hace_un_año hasta hoy
    ultimos_365_dias = xmex.sessions_in_range(dia_habil_hace_un_año, hoy)

    resultado['mes_hasta_fecha'] = mes_hasta_fecha
    resultado['año_hasta_fecha'] = año_hasta_fecha
    resultado['ultimos_30_dias'] = ultimos_30_dias
    resultado['ultimos_90_dias'] = ultimos_90_dias
    resultado['ultimos_180_dias'] = ultimos_180_dias
    resultado['ultimos_365_dias'] = ultimos_365_dias

    return resultado

Definiremos dos funciones, una para obtener los rendimientos anualizados y otra para los rendimientos brutos para cada activo.

In [ ]:
def anualizar_rendimiento(rendimiento_bruto, dias):
    rendimiento_anualizado = rendimiento_bruto / dias * 360
    return rendimiento_anualizado

def calcular_rendimiento_bruto(precio_inicio, precio_fin, dias):
    # Calcular el cambio porcentual en el precio
    cambio_pct = (precio_fin / precio_inicio) - 1

    # Calcular el rendimiento bruto
    rendimiento_bruto = cambio_pct
    return rendimiento_bruto

Definiremos una funcion para obtener los rendimientos de cada activo utilizando las ventanas de tiempo y la anualización de rendimientos ya planteados.

In [ ]:
def calcular_rendimiento(precios, ventanas_de_tiempo, nombre_benchmark):
    rendimientos = []

    for periodo, ventana in ventanas_de_tiempo.items():
        # Obtén los precios de inicio y fin para la ventana de tiempo actual
        precio_inicio = precios.loc[ventana[0], nombre_benchmark]
        precio_fin = precios.loc[ventana[-1], nombre_benchmark]

        # Calcula el rendimiento bruto y anualizado
        rendimiento_bruto = calcular_rendimiento_bruto(precio_inicio, precio_fin, (ventana[-1] - ventana[0]).days)
        rendimiento_anualizado = anualizar_rendimiento(rendimiento_bruto, (ventana[-1] - ventana[0]).days)

        # Agrega el rendimiento a la lista de rendimientos
        rendimientos.append({
            'Periodo': periodo,
            'Rendimiento_bruto': rendimiento_bruto*100,
            'Rendimiento_anualizado': rendimiento_anualizado*100
        })

    # Convierte la lista de rendimientos en un dataframe
    df_rendimientos = pd.DataFrame(rendimientos)

    return df_rendimientos

Definimos una fecha limite para nuestras ventanas.

In [ ]:
hoy = pd.Timestamp("2023-11-30")

ventanas = calcular_fechas(hoy)

Definimos una función para obtener cada una de las tablas de los rendimientos de cada activo.

In [ ]:
df_rendimientos_1 = calcular_rendimiento(df, ventanas, etfs[0])
df_rendimientos_1

,Periodo,Rendimiento_bruto,Rendimiento_anualizado
0,mes_hasta_fecha,-0.142711,-1.712531
1,año_hasta_fecha,-11.845644,-12.729648
2,ultimos_30_dias,-0.523899,-6.083987
3,ultimos_90_dias,2.529271,9.897148
4,ultimos_180_dias,-3.755369,-7.347462
5,ultimos_365_dias,-11.991181,-11.826919


In [ ]:
df_rendimientos_2 = calcular_rendimiento(df, ventanas, etfs[1])
df_rendimientos_2

,Periodo,Rendimiento_bruto,Rendimiento_anualizado
0,mes_hasta_fecha,1.149664,13.795964
1,año_hasta_fecha,-10.901658,-11.715214
2,ultimos_30_dias,1.652404,19.189208
3,ultimos_90_dias,3.844372,15.043194
4,ultimos_180_dias,-0.124927,-0.244423
5,ultimos_365_dias,-10.192395,-10.052774


In [ ]:
df_rendimientos_3 = calcular_rendimiento(df, ventanas, etfs[2])
df_rendimientos_3

,Periodo,Rendimiento_bruto,Rendimiento_anualizado
0,mes_hasta_fecha,8.269727,99.236730
1,año_hasta_fecha,39.834331,42.807042
2,ultimos_30_dias,8.575565,99.587208
3,ultimos_90_dias,8.849357,34.627920
4,ultimos_180_dias,10.969551,21.462166
5,ultimos_365_dias,29.433140,29.029946


In [ ]:
df_rendimientos_4 = calcular_rendimiento(df, ventanas, etfs[3])
df_rendimientos_4

,Periodo,Rendimiento_bruto,Rendimiento_anualizado
0,mes_hasta_fecha,4.235645,50.827734
1,año_hasta_fecha,2.433832,2.615461
2,ultimos_30_dias,4.411833,51.234191
3,ultimos_90_dias,4.357067,17.049394
4,ultimos_180_dias,8.750542,17.120626
5,ultimos_365_dias,2.764547,2.726677


In [ ]:
df_rendimientos_5 = calcular_rendimiento(df, ventanas, etfs[4])
df_rendimientos_5

,Periodo,Rendimiento_bruto,Rendimiento_anualizado
0,mes_hasta_fecha,-8.446135,-101.353622
1,año_hasta_fecha,-13.478226,-14.484064
2,ultimos_30_dias,-9.114505,-105.845861
3,ultimos_90_dias,-1.063379,-4.161047
4,ultimos_180_dias,7.681083,15.028207
5,ultimos_365_dias,-13.922914,-13.732189


# **Estadisticas de Activos**: Medidas de Riesgo

### **Media**

In [ ]:
# Calculo de la media de los activos
mean = returns.mean()
mean

AGG       0.000106
GSG       0.000041
IEMS.L    0.000240
IYW       0.000805
SEMB.L    0.000094
dtype: float64

### **Sesgo**

In [ ]:
#Calculo para Sesgo de los activos
sesgo = returns.skew()
sesgo

AGG       0.768364
GSG      -0.133517
IEMS.L   -0.210215
IYW       0.402074
SEMB.L    1.352046
dtype: float64

### **Exceso de Curtosis**

In [ ]:
#Calculo para Exceso de Curtosis:
curtosis = returns.kurtosis()
curtosis

AGG        5.447596
GSG        4.858784
IEMS.L     9.728775
IYW       10.738301
SEMB.L    95.234380
dtype: float64

### **VaR Parametrico**

In [ ]:
#Calculo para VaR Parametrico
mean = np.mean(returns)
stdev = returns.std()

VaR_95=norm.ppf(1-0.95,mean,stdev)
print(VaR_95)


[-0.0138899  -0.02279335 -0.01982247 -0.02329927 -0.01728506]


### **VaR Historico**

In [ ]:
#Calculo de VaR Historico (Necesario para el CVaR)
hVaR_95 = returns.quantile(0.05)
hVaR_95

AGG      -0.012498
GSG      -0.021349
IEMS.L   -0.017874
IYW      -0.021836
SEMB.L   -0.014263
Name: 0.05, dtype: float64

### **CVaR**

In [ ]:
#Calculo CVaR
CVaR_95=returns[returns<=hVaR_95].mean()
print(CVaR_95)

AGG      -0.017367
GSG      -0.031123
IEMS.L   -0.026937
IYW      -0.032895
SEMB.L   -0.020704
dtype: float64


### **Sharpe Ratio**

In [ ]:
# Tasa de rendimiento del bono de 1 año (anualizada y en formato decimal)
Rf = 0.05426 / 252  # Dividido por 252 para obtener la tasa diaria

# Calcular el rendimiento medio diario y la desviación estándar de los rendimientos
Rp_mean = returns.mean()
sigma_p = returns.std()

# Calcular el Sharpe Ratio
sharpe_ratio = (Rp_mean - Rf) / sigma_p
sharpe_ratio

AGG      -0.012819
GSG      -0.012539
IEMS.L    0.002060
IYW       0.040212
SEMB.L   -0.011447
dtype: float64

### **Sortino**

In [ ]:
# Calcular la desviación estándar de los rendimientos negativos usando la serie
negative_returns = returns[returns < Rf]
sigma_dp_series = negative_returns.std()

# Calcular el Sortino Ratio usando la serie
sortino_ratio_series = (returns.mean() - Rf) / sigma_dp_series
sortino_ratio_series

AGG      -0.021411
GSG      -0.017987
IEMS.L    0.002869
IYW       0.057391
SEMB.L   -0.016346
dtype: float64

### **Drowdowns**

In [ ]:
# Calcular la serie acumulativa de rendimientos
cumulative_returns = (1 + returns).cumprod()

# Calcular el pico acumulativo previo en cada punto del tiempo
running_max = cumulative_returns.cummax()

# Calcular la serie de drawdowns
drawdowns = (cumulative_returns - running_max) / running_max

# Encontrar el drawdown máximo
max_drawdown = drawdowns.min()

max_drawdown, drawdowns.tail()

(AGG      -0.448542
 GSG      -0.590657
 IEMS.L   -0.381493
 IYW      -0.442657
 SEMB.L   -0.453343
 dtype: float64,
                  AGG       GSG    IEMS.L       IYW    SEMB.L
 Date                                                        
 2023-12-04 -0.431957 -0.365498 -0.249679 -0.200159 -0.429120
 2023-12-05 -0.419361 -0.360458 -0.240526 -0.183408 -0.417735
 2023-12-06 -0.421061 -0.381618 -0.240409 -0.195537 -0.419903
 2023-12-07 -0.423246 -0.381316 -0.243418 -0.185181 -0.422713
 2023-12-08 -0.420021 -0.368048 -0.241901 -0.170533 -0.417167)

# **Portafolios Optimos**: Datos Necesarios

In [ ]:
def download_datos(tickers, start_date='2010-01-01', end_date='2021-01-01'):
    datos = yf.download(tickers, start=start_date, end=end_date)

    return datos['Close']

In [ ]:
#Definimos nuestro conjunto de datos.
datos = download_datos(symbols)
datos = datos.dropna()
#Pasando a pesos todo
datos["AGG"] = datos["AGG"] * datos["MXN=X"]
datos["SEMB.L"] = datos["SEMB.L"] * datos["GBPMXN=X"]
datos["IEMS.L"] = datos["IEMS.L"] * datos["MXN=X"]
datos["IYW"] = datos["IYW"] * datos["MXN=X"]
datos["GSG"] = datos["GSG"] * datos["MXN=X"]
datos.tail()

[*********************100%%**********************]  7 of 7 completed


,AGG,GBPMXN=X,GSG,IEMS.L,IYW,MXN=X,SEMB.L
Date,,,,,,,
2020-12-23,2369.835868,26.936899,243.859094,1502.859500,1705.204295,20.103800,229111.796020
2020-12-24,2365.569739,27.074997,244.195615,1509.481745,1707.765360,20.048901,228946.174194
2020-12-29,2361.337708,26.937500,242.875886,1535.679822,1707.133382,20.006250,230005.843750
2020-12-30,2354.323407,26.907400,243.605699,1543.965648,1695.870398,19.934999,228363.104914
2020-12-31,2350.905479,27.108900,245.254791,1540.251766,1692.317779,19.890900,229653.046945


In [ ]:
#Obtenemos los retornos en el periodo definido por el ejercicio
retornos = datos.pct_change().fillna(0)
retornos = retornos.drop( tasasdecambio ,axis = 1)
retornos.tail()

,AGG,GSG,IEMS.L,IYW,SEMB.L
Date,,,,,
2020-12-23,0.005393,0.019523,0.017096,-0.001342,-0.009202
2020-12-24,-0.001800,0.001380,0.004406,0.001502,-0.000723
2020-12-29,-0.001789,-0.005404,0.017356,-0.000370,0.004628
2020-12-30,-0.002970,0.003005,0.005396,-0.006598,-0.007142
2020-12-31,-0.001452,0.006770,-0.002405,-0.002095,0.005649


In [ ]:
#Definimos una función para simular diferentes combinaciones de carteras y evaluar su rendimiento.
def portfolio_simulation(returns):

    # Initialize the lists
    rets = []; vols = []; wts = []

    # Simulate 5,000 portfolios
    for i in range (numofportfolio):

        # Generate random weights
        weights = random.random(numofasset2)[:, newaxis]

        # Set weights such that sum of weights equals 1
        weights /= sum(weights)

        # Portfolio statistics
        rets.append(weights.T @ array(retornos.mean() * 252)[:, newaxis])
        vols.append(sqrt(multi_dot([weights.T, retornos.cov()*252, weights])))
        wts.append(weights.flatten())

    # Create a dataframe for analysis
    portdf = 100*pd.DataFrame({
        'port_rets': array(rets).flatten(),
        'port_vols': array(vols).flatten(),
        'weights': list(array(wts))
        })

    portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']

    return round(portdf,2)

In [ ]:
# Create a dataframe for analysis
temp = portfolio_simulation(retornos)
temp.head()

,port_rets,port_vols,weights,sharpe_ratio
0,4.58,12.11,"[20.878265603058573, 24.494952286646456, 32.97...",0.38
1,9.31,11.75,"[38.483483506909444, 5.8772229316598175, 20.59...",0.79
2,9.29,11.86,"[24.64442716976253, 15.75060070334209, 8.90507...",0.78
3,12.02,14.38,"[8.412460797777758, 13.083357345964973, 24.542...",0.84
4,6.74,13.26,"[27.329192798676498, 21.126281916656726, 36.67...",0.51


In [ ]:
# Get the max sharpe portfolio stats
temp.iloc[temp.sharpe_ratio.idxmax()]

port_rets                                                   15.58
port_vols                                                   15.18
weights         [4.734064056406238, 5.148564881902852, 1.91904...
sharpe_ratio                                                 1.03
Name: 87, dtype: object

In [ ]:
# Verify the above result
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
port_rets,1000.0,8.07685,2.414361,1.11,6.50,8.08,9.5925,17.32
port_vols,1000.0,12.76854,1.102218,10.74,11.97,12.66,13.3500,17.78
sharpe_ratio,1000.0,0.63196,0.173828,0.07,0.52,0.64,0.7600,1.03


In [ ]:
# Max sharpe ratio portfolio weights
msrpwts = temp['weights'][temp['sharpe_ratio'].idxmax()]

# Allocation to achieve max sharpe ratio portfolio
dict(zip(symbols, around(msrpwts,2)))

{'AGG': 4.73, 'SEMB.L': 5.15, 'IYW': 1.92, 'IEMS.L': 59.38, 'GSG': 28.82}

In [ ]:
# Define portfolio stats function
def portfolio_stats(weights):

    weights = array(weights)[:,newaxis]
    port_rets = weights.T @ array(returns.mean() * 252)[:,newaxis]
    port_vols = sqrt(multi_dot([weights.T, returns.cov() * 252, weights]))

    return array([port_rets, port_vols, port_rets/port_vols]).flatten()

In [ ]:
# Definimos un portafolio de pesos equitativos.
wts = numofasset2 * [1./numofasset2]
array(wts).shape

(5,)

In [ ]:
# Tomando los pesos
wts = numofasset2 * [1./numofasset2]
wts = array(wts)[:,newaxis]
wts

array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])

In [ ]:
# Revisando el shape
wts.shape

(5, 1)

In [ ]:
#
ret = array(retornos.mean() * 252)[:,newaxis]
ret

array([[ 0.06220394],
       [-0.02869543],
       [ 0.07741893],
       [ 0.22690009],
       [ 0.06297225]])

In [ ]:
# Check shape
ret.shape

(5, 1)

# **Maximo Sharpe Ratio**

In [ ]:
# Maximizing sharpe ratio
def min_sharpe_ratio(weights):
    return -portfolio_stats(weights)[2]

In [ ]:
# Each asset boundary ranges from 0 to 1
tuple((0, 1) for x in range(numofasset))

((0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1))

In [ ]:
# Specify constraints and bounds
cons = ({'type': 'eq', 'fun': lambda x: sum(x) - 1})
bnds = tuple((0, 1) for x in range(numofasset2))
initial_wts = numofasset2*[1./numofasset2]

In [ ]:
# Optimizing for maximum sharpe ratio
port_sharpe = sco.minimize(min_sharpe_ratio, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)
port_sharpe

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.8716018741778371
       x: [ 1.941e-17  0.000e+00  1.908e-17  1.000e+00  2.677e-17]
     nit: 3
     jac: [ 4.128e-02  2.440e-01  1.241e-01 -7.451e-09  2.320e-02]
    nfev: 18
    njev: 3

In [ ]:
# Portfolio weights
list(zip(symbols, around(port_sharpe['x']*100,2)))

[('AGG', 0.0), ('SEMB.L', 0.0), ('IYW', 0.0), ('IEMS.L', 100.0), ('GSG', 0.0)]

In [ ]:
# Portfolio stats
stats = ['Returns', 'Volatility', 'Sharpe Ratio']
list(zip(stats, around(portfolio_stats(port_sharpe['x']),4)))

[('Returns', 0.2028), ('Volatility', 0.2326), ('Sharpe Ratio', 0.8716)]

# **Minima Volatidad**

In [ ]:
# Minimize the variance
def min_variance(weights):
    return portfolio_stats(weights)[1]**2

In [ ]:
port_min_var = sco.minimize(min_variance, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)
port_min_var

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.012438439221144095
       x: [ 4.543e-01  1.227e-01  1.232e-01  7.605e-03  2.921e-01]
     nit: 8
     jac: [ 2.479e-02  2.537e-02  2.497e-02  2.448e-02  2.478e-02]
    nfev: 48
    njev: 8

In [ ]:
# Portfolio weights
list(zip(symbols, around(port_min_var['x']*100,2)))

[('AGG', 45.43),
 ('SEMB.L', 12.27),
 ('IYW', 12.32),
 ('IEMS.L', 0.76),
 ('GSG', 29.21)]

In [ ]:
# Portfolio stats
list(zip(stats, around(portfolio_stats(port_min_var['x']),4)))

[('Returns', 0.0294), ('Volatility', 0.1115), ('Sharpe Ratio', 0.2636)]

# **Minima Volatilidad y 10% de Rendimiento**

In [ ]:
#Queremos rendimiento 10% anual
rendimiendo_deseado  =0.1

In [ ]:
#Matriz de covarianza de los rendimientos de los activos
matrix_cov = retornos.cov()
matrix_cov


,AGG,GSG,IEMS.L,IYW,SEMB.L
AGG,0.000075,0.000032,0.000032,0.000032,0.000022
GSG,0.000032,0.000170,0.000065,0.000072,0.000014
IEMS.L,0.000032,0.000065,0.000146,0.000088,0.000020
IYW,0.000032,0.000072,0.000088,0.000184,0.000026
SEMB.L,0.000022,0.000014,0.000020,0.000026,0.000086


In [ ]:
#Generemos la restricciones
#Restricción 1: Pesos iagual a 1
#Restricción 2: Rendimiento igual a 10%


cons  =({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # Suma de pesos igual a 1
               {'type': 'eq', 'fun': lambda weights: np.dot(weights, ret) - rendimiendo_deseado})  # Rendimiento objetivo


In [ ]:
#Función para calcualr la varianza del portafolio
def objective_function(weights, mean_returns, matrix_cov):
    portfolio_variance = np.dot(weights.T, np.dot(matrix_cov*252, weights))
    return portfolio_variance

In [ ]:
port_min_var_rendimiento = minimize(objective_function, initial_wts, args=(ret, matrix_cov),method='SLSQP', bounds=bnds, constraints=cons)

#Pesos
port_min_var_rendimiento.x

array([0.35659231, 0.        , 0.05508988, 0.22269466, 0.36562314])

In [ ]:
#Suma de los pesos
sum(port_min_var_rendimiento.x)

1.0

In [ ]:
#Rendimiento
np.dot(port_min_var_rendimiento.x ,ret)

array([0.1])

# **BackTesting**:

In [ ]:
#Vamos a bajar los datos en el intervalo dado:
def download_datos(tickers, start_date='2021-01-01', end_date=date.today().strftime('%Y-%m-%d')):
    datos = yf.download(tickers, start=start_date, end=end_date)

    return datos['Close']

In [ ]:
#Bajamos los datos de nuestros trickers
symbols = ['AGG', 'SEMB.L', 'IYW', 'IEMS.L', 'GSG', 'MXN=X', 'GBPMXN=X' , '^GSPC' ]
df=download_datos(symbols)
df = df.dropna()

[*********************100%%**********************]  8 of 8 completed


In [ ]:
#Pasando a pesos todo
df["AGG"] = df["AGG"] * df["MXN=X"]
df["SEMB.L"] = df["SEMB.L"] * df["GBPMXN=X"]
df["IEMS.L"] = df["IEMS.L"] * df["MXN=X"]
df["IYW"] = df["IYW"] * df["MXN=X"]
df["GSG"] = df["GSG"] * df["MXN=X"]
df["GSG"] = df["^GSPC"] * df["MXN=X"]

In [ ]:
#Calculando Retornos
returns = df.pct_change().fillna(0)
extras = ["GBPMXN=X", "MXN=X", '^GSPC']
returns = returns.drop(extras, axis = 1)

#Son los retornos medios del 2021 al 2023
retornos_medios = returns.mean()

## **Calculando el rendimiento anual de cada portafolio óptimo calculado en el paso C**

In [ ]:
#Retorno del portafolio minima volatilidad
Return1 = port_min_var.x .T@ ret
Return1

array([0.05440396])

In [ ]:
#Retorno del portafolio minima volatilidad con 10% de rendimiento anual
Return2 = port_min_var_rendimiento.x.T@ret
Return2
#Se cumple con lo pedido en el paso anterior

array([0.1])

In [ ]:
#Retorno del portafolio Maximo Sharpe Ratio
Return3 = port_sharpe.x.T @ ret
Return3

array([0.22690009])

In [ ]:
#Retorno del portafolio equitativo
Return4 = wts.T@ret
Return4

array([[0.08015996]])

In [ ]:
#Calculando retornos para S&P500
sp500 = df['^GSPC'].pct_change().fillna(0)
#Calculando retornos medios para S&P500
sp500_ret_medios = sp500.mean()
sp500_ret_medios

0.0003659540472951959

### **Obtengamos los rendimientos de nuestros portafolios óptimos**

In [ ]:

#Creamos un data frame auxiliar para ultiplicar nuestros rendimientos por los pesos correspondientes
returns_min_var = returns.copy()
returns_max_sharpe = returns.copy()
returns_min_var_rendimientos = returns.copy()
returns_equidad = returns.copy()

#Mi lista de ETFs
nombre  =['AGG','GSG','IEMS.L','IYW','SEMB.L']


In [ ]:

#Obtenemos el data
#Sustituimos el data frame multiplicando los retornos por los pesos
for i in range(5):
  returns_min_var[nombre[i]]=returns_min_var[nombre[i]]*port_min_var.x[i] #Creación del data frame usando minima volatilidadad
  returns_max_sharpe[nombre[i]]=returns_max_sharpe[nombre[i]]*port_sharpe.x[i] #Creación del data frame usando maximo sharpe ratio
  returns_min_var_rendimientos[nombre[i]]=returns_min_var_rendimientos[nombre[i]]*port_min_var_rendimiento.x[i] #Creación del data frame usando minima volatilidadad con 10% rendimiento anual
  returns_equidad[nombre[i]]=returns_equidad[nombre[i]]*wts[i] #Creación del data frame usando pesos iguales


In [ ]:
#Sumamos los rendimeintos diarios
returns_min_var_Final = returns_min_var[nombre[0]] + returns_min_var[nombre[1]] + returns_min_var[nombre[2]] + returns_min_var[nombre[3]] + returns_min_var[nombre[4]]
returns_max_sharpe_Final = returns_max_sharpe[nombre[0]] + returns_max_sharpe[nombre[1]] + returns_max_sharpe[nombre[2]] + returns_max_sharpe[nombre[3]] + returns_max_sharpe[nombre[4]]
returns_min_var_rendimientos_Final = returns_min_var_rendimientos[nombre[0]] + returns_min_var_rendimientos[nombre[1]] + returns_min_var_rendimientos[nombre[2]] + returns_min_var_rendimientos[nombre[3]] + returns_min_var_rendimientos[nombre[4]]
returns_equidad_Final = returns_equidad[nombre[0]] + returns_equidad[nombre[1]] + returns_equidad[nombre[2]] + returns_equidad[nombre[3]] + returns_equidad[nombre[4]]


### **MÉTRICAS DE RIESGO DE NUESTROS PORTAFOLIOS ÓPTIMOS**

In [ ]:
#Sesgo
sesgo_port1 = returns_min_var_Final.skew()
sesgo_port2 = returns_max_sharpe_Final.skew()
sesgo_port3 = returns_min_var_rendimientos_Final.skew()
sesgo_port4 = returns_equidad_Final.skew()
sesgo_sp500 = sp500.skew()

sesgo_general = [sesgo_port1,sesgo_port2,sesgo_port3,sesgo_port4,sesgo_sp500]

In [ ]:
#kurtosis
kurt_port1 = kurtosis(returns_min_var_Final)
kurt_port2 = kurtosis(returns_max_sharpe_Final)
kurt_port3 = kurtosis(returns_min_var_rendimientos_Final)
kurt_port4 = kurtosis(returns_equidad_Final)
kurt_sp500 = kurtosis(sp500)

kurtosis_general = [kurt_port1,kurt_port2,kurt_port3,kurt_port4,kurt_sp500]

### **VaR**

In [ ]:
# Parametric VaR

VaR_95_port1 = norm.ppf(1-0.95,np.mean(returns_min_var_Final),np.std(returns_min_var_Final))
VaR_95_port2 = norm.ppf(1-0.95,np.mean(returns_max_sharpe_Final),np.std(returns_max_sharpe_Final))
VaR_95_port3 = norm.ppf(1-0.95,np.mean(returns_min_var_rendimientos_Final),np.std(returns_min_var_rendimientos_Final))
VaR_95_port4 = norm.ppf(1-0.95,np.mean(returns_equidad_Final),np.std(returns_equidad_Final))
VaR_95_sp500 = norm.ppf(1-0.95,np.mean(sp500),np.std(sp500))

VaR_95_general = [VaR_95_port1,VaR_95_port2,VaR_95_port3,VaR_95_port4,VaR_95_sp500]

### **Historical VaR**

In [ ]:
# Historical VaR
hVaR_95_prt1 = returns_min_var_Final.quantile(0.05)
hVaR_95_prt2 = returns_max_sharpe_Final.quantile(0.05)
hVaR_95_prt3 = returns_min_var_rendimientos_Final.quantile(0.05)
hVaR_95_prt4 = returns_equidad_Final.quantile(0.05)
hVaR_95_sp500 = sp500.quantile(0.05)

hVaR_general = [hVaR_95_prt1,hVaR_95_prt2,hVaR_95_prt3,hVaR_95_prt4,hVaR_95_sp500]

### **CVaR**

In [ ]:
CVaR_95_port1 = returns_min_var_Final[returns_min_var_Final<=hVaR_95_prt1].mean()
CVaR_95_port2 = returns_max_sharpe_Final[returns_max_sharpe_Final<=hVaR_95_prt2].mean()
CVaR_95_port3 = returns_min_var_rendimientos_Final[returns_min_var_rendimientos_Final<=hVaR_95_prt3].mean()
CVaR_95_port4 = returns_equidad_Final[returns_equidad_Final<=hVaR_95_prt4].mean()
CVaR_95_sp500 = sp500[sp500<=hVaR_95_prt4].mean()

CVaR_95_general = [CVaR_95_port1,CVaR_95_port2,CVaR_95_port3,CVaR_95_port4,CVaR_95_sp500]

### **Sharpe Ratio**

In [ ]:
# Tasa de rendimiento del bono de 1 año (anualizada y en formato decimal)
Rf = 0.05426

# Calcular el rendimiento medio diario y la desviación estándar de los rendimientos
Rp_mean_port1 = returns_min_var_Final.mean() * 252
Rp_mean_port2 = returns_max_sharpe_Final.mean() * 252
Rp_mean_port3 = returns_min_var_rendimientos_Final.mean() * 252
Rp_mean_port4 = returns_equidad_Final.mean() * 252
Rp_mean_sp500 = sp500.mean() * 252


sigma_p_port1= returns_min_var_Final.std() * np.sqrt(252)
sigma_p_port2= returns_max_sharpe_Final.std() * np.sqrt(252)
sigma_p_port3= returns_min_var_rendimientos_Final.std() * np.sqrt(252)
sigma_p_port4= returns_equidad_Final.std() * np.sqrt(252)
sigma_p_sp500= sp500.std() * np.sqrt(252)


# Calcular el Sharpe Ratio
sharpe_ratio_port1 = (Rp_mean_port1 - Rf) / sigma_p_port1
sharpe_ratio_port2 = (Rp_mean_port2 - Rf) / sigma_p_port2
sharpe_ratio_port3 = (Rp_mean_port3 - Rf) / sigma_p_port3
sharpe_ratio_port4 = (Rp_mean_port4 - Rf) / sigma_p_port4
sharpe_ratio_sp500 = (Rp_mean_sp500 - Rf) / sigma_p_sp500


sharpe_ratio_general = [sharpe_ratio_port1,sharpe_ratio_port2,sharpe_ratio_port3,sharpe_ratio_port4,sharpe_ratio_sp500]

### **Sortino**

In [ ]:
# Calcular la desviación estándar de los rendimientos negativos usando la serie
negative_returns_port1 = returns_min_var_Final[returns_min_var_Final < Rf]
negative_returns_port2 = returns_max_sharpe_Final[returns_max_sharpe_Final < Rf]
negative_returns_port3 = returns_min_var_rendimientos_Final[returns_min_var_rendimientos_Final < Rf]
negative_returns_port4 = returns_equidad_Final[returns_equidad_Final < Rf]
negative_returns_sp500 = sp500[sp500 < Rf]



sigma_dp_series_port1 = negative_returns_port1.std()
sigma_dp_series_port2 = negative_returns_port2.std()
sigma_dp_series_port3 = negative_returns_port3.std()
sigma_dp_series_port4 = negative_returns_port4.std()
sigma_dp_series_sp500 = negative_returns_sp500.std()


# Calcular el Sortino Ratio usando la serie
sortino_ratio_series_port1 = (returns_min_var_Final.mean() - Rf) / sigma_dp_series_port1
sortino_ratio_series_port2 = (returns_max_sharpe_Final.mean() - Rf) / sigma_dp_series_port2
sortino_ratio_series_port3 = (returns_min_var_rendimientos_Final.mean() - Rf) / sigma_dp_series_port3
sortino_ratio_series_port4 = (returns_equidad_Final.mean() - Rf) / sigma_dp_series_port4
sortino_ratio_series_sp500 = (sp500 .mean() - Rf) / sigma_dp_series_sp500

sortino_ratio_general = [sortino_ratio_series_port1,sortino_ratio_series_port2,sortino_ratio_series_port3,sortino_ratio_series_port4,sortino_ratio_series_sp500]

### **Drawdowns**

In [ ]:
# Calcular la serie acumulativa de rendimientos
cumulative_returns_port1 = (1 + returns_min_var_Final).cumprod()
cumulative_returns_port2 = (1 + returns_max_sharpe_Final).cumprod()
cumulative_returns_port3 = (1 + returns_min_var_rendimientos_Final).cumprod()
cumulative_returns_port4 = (1 + returns_equidad_Final).cumprod()
cumulative_returns_sp500 = (1 + sp500).cumprod()


# Calcular el pico acumulativo previo en cada punto del tiempo
running_max_port1 = cumulative_returns_port1.cummax()
running_max_port2 = cumulative_returns_port2.cummax()
running_max_port3 = cumulative_returns_port3.cummax()
running_max_port4 = cumulative_returns_port4.cummax()
running_max_sp500 = cumulative_returns_sp500.cummax()



# Calcular la serie de drawdowns
drawdowns_port1 = (cumulative_returns_port1 - running_max_port1) / running_max_port1
drawdowns_port2 = (cumulative_returns_port2 - running_max_port2) / running_max_port2
drawdowns_port3 = (cumulative_returns_port3 - running_max_port3) / running_max_port3
drawdowns_port4 = (cumulative_returns_port4 - running_max_port4) / running_max_port4
drawdowns_sp500 = (cumulative_returns_sp500 - running_max_sp500) / running_max_sp500


# Encontrar el drawdown máximo
max_drawdown_port1 = drawdowns_port1.min()
max_drawdown_port2 = drawdowns_port2.min()
max_drawdown_port3 = drawdowns_port3.min()
max_drawdown_port4 = drawdowns_port4.min()
max_drawdown_sp500 = drawdowns_sp500.min()

drawdown_general = [max_drawdown_port1,max_drawdown_port2,max_drawdown_port3,max_drawdown_port4,max_drawdown_sp500]

###**Nota Importante:**
Se mostrarán los arrays de la siguiente manera.\
1.-Portafolio Óptimo de Mínima Volatilidad.\
2.-Portafolio Óptimo Máximo Sharpe Ratio.\
3.-Portafolio Óptimo Mínima Volatilidad con 10% de rendimiento anual.\
4.-Portafolio Equitativo.\
5.- Índice S&P500.


In [ ]:
sesgo_general

[0.49759892752176627,
 0.09096714881419593,
 0.5710957612676159,
 0.08511706334458345,
 -0.13788531375427387]

### **Analizando el sesgo de los portafolios**
Notemos que el Portafolio Mínima volatilidad (1) y Portafolio de Minima volatilidad con rendimineto del 10% (3) se encuenta con valores positivos altos, esto nos quiere decir que sus rendimientos fueron más elevados que la media, en cambio S&P500 muestra un valor negativo, no es considerablemente alto pero nos indica que sus retornos se mantuvieron por debajo de la media.

El Portafolio Maximo Sharpe Ratio(2) y Portafolio Equitativo(4), se mantuvieron
muy cercanos a la media.

In [ ]:
kurtosis_general

[10.052346466794242,
 1.134385342561731,
 14.33801662508569,
 2.6890224816637396,
 1.5489963928683688]

### **Analizando el exceso de Kurtosis**
Notemos que el Portafolio 1 y 3 tiene Kurtosis altas, lo que nos indica que estos ETFs van a tener rendimientos altos, lo cual complemento a lo obtenido en el sesgo (mayores a la media), y en los demás portafolios se tienen valores cercanos a 0, por lo que sus rendimientos no serán tan altos como los dos anteriores

In [ ]:
VaR_95_general

[-0.013827662068005994,
 -0.02934127148199311,
 -0.01563260838749728,
 -0.016297275726713814,
 -0.018129986288309665]

### **Analizando el VaR**
Todos los portafolios no exceden el 3% de perdidás del valor del portafolio, lo cual puede indicar que nuestra exposición al riesgo en los portafolios no es tan alta, los portafolios de mínima volatilidad (1 y 3) contienen la menor cantidad de exposición de riesgo de los 5 portafolios.

In [ ]:
CVaR_95_general

[-0.016655648615992138,
 -0.03839851528542282,
 -0.019388077932689474,
 -0.020834254962583957,
 -0.022471833152549518]

### **Analizando el CVaR**
El CVaR nos proporcia el promedio de las perdidas en caso de sobrepasar el VaR, notemos que nuestros 5 portafolios se mantienen por debajo del 5% de pérdida, notemos que si elegimos el portafolio Maximo Sharpe Ratio(2) estamos eligiendo el portafolio más expuesto al riesgo, ya que es el máximo en pérdidas tanto en el VaR como en CVaR, los portafolios de Minima volatilidad y Mínima volatilidad siguen sinedo los portafolios con menos pérdidas de los 5.

In [ ]:
sharpe_ratio_general

[-1.0128145369631703,
 0.2208052323888032,
 -0.7513804607137871,
 -0.43143178435460133,
 0.21251060717326098]

### **Analizando el Sharpe Ratio**
Los portafolios de mínima volatilidad (1 y 3) no son tan buenos cuando se habla de riesgo-beneficio, ya que son los valores más negativos de mis 5 muestras.

Los mejores portafolios para riesgo-beneficio son Máximo Sharpe Ratio y S&P500, lo que nos indica que superan la tasa libre de riesgo, pero no son tan altos.


In [ ]:
sortino_ratio_general

[-6.966434410453034,
 -3.0365487300683522,
 -6.177203074307726,
 -5.49590173276889,
 -4.867829008232959]

### **Analizando Sortino**
Nuestros porfatolios y el S&P500 al ser negativos en la metrica de riesgo Sortino nos indica que hay hay pérdidas desfavorables en relación a la tasa de rendimiento esperado.

In [ ]:
#Drardowns
drawdown_general

[-0.3409715086391202,
 -0.44265712174991634,
 -0.3391717143751583,
 -0.32098580326279635,
 -0.2537811304072361]

### **Analizando Drawdown**
Los portafolios tuvieron una gran disminución entre su alto historico con su minimo historico, siendo el de Máximo Sharpe Ratio el que tuvo la caída más grande, en general fueron caídas grandes.

Podemos notar que los protafolios de Miníma volatilidad cumplen que son los portafoliso con menos pérdidas esperadas pero su riesgo-beneficio no estan buena.

Máximo Sharpe Ratio es el portafolio más expuesto al riesgo según lo obtenido en el VaR y CVaR, pero la Métrica Sharpe Ratio nos indica que ese riesgo que se toma nos da más beneficio que los demás, además fue el portafolio que tuvo la fructuación más grande entre el máximo historico y el mínimo historico por lo visto en Drawdown.

El equitativo muestra datos curiosos, al ser un portafolio equitativo, no hubo restricciones en cuanto a minimizar volatilidad o obtener cierto rendimiento, este tuvo valores cercanos a su media, y obteniendo VaR y CVaR menores a otros portafolios, claro que su riesgo-beneficio no es buneo obteniendo un número negativo relativamente alto respecto a otros.



**Rendimientos Anuales y Acumulados**

In [ ]:
def anualizar_rendimiento(rendimiento_bruto, dias):
    rendimiento_anualizado = rendimiento_bruto / dias * 360
    return rendimiento_anualizado

def calcular_rendimiento_bruto(precio_inicio, precio_fin, dias):
    # Calcular el cambio porcentual en el precio
    cambio_pct = (precio_fin / precio_inicio) - 1

    # Calcular el rendimiento bruto
    rendimiento_bruto = cambio_pct
    return rendimiento_bruto

In [ ]:
rend = pd.concat([returns_min_var_Final, returns_max_sharpe_Final, returns_min_var_rendimientos_Final,returns_equidad_Final, sp500], axis = 1)
rend.columns = ["Min_Var","Max_SR","Min_Var10","Equitativos", "S&P500"]

dias = returns_min_var_Final.shape[0]

In [ ]:
#Rendimietnto Bruto Min_Var
rendb_MV = calcular_rendimiento_bruto(rend["Min_Var"].values[1], rend["Min_Var"].iloc[-1], dias)
rendb_MV

6.692746480108182

In [ ]:
#Rendimientos Acumulado Min_Var
anualizar_rendimiento(rendb_MV,252)

9.561066400154546

In [ ]:
#Rendimientos Bruto Max_SR
rendb_MSR = calcular_rendimiento_bruto(rend["Max_SR"].values[1], rend["Max_SR"].iloc[-1], dias)
rendb_MSR

0.3897609820306265

In [ ]:
#Rendimientos Acumulado Max_SR
anualizar_rendimiento(rendb_MSR,252)

0.556801402900895

In [ ]:
#Rendimientos Bruto "Min_Var10"
rendb_MV10 = calcular_rendimiento_bruto(rend["Min_Var10"].values[1], rend["Min_Var10"].iloc[-1], dias)
rendb_MV10

10.692731572903194

In [ ]:
#Rendimientos Acumulado Min_Var10
anualizar_rendimiento(rendb_MV10,252)

15.275330818433135

In [ ]:
#Rendimientos Bruto "Equitativos"
rendb_E = calcular_rendimiento_bruto(rend["Equitativos"].values[1], rend["Equitativos"].iloc[-1], dias)
rendb_E

1.091361040100737

In [ ]:
#Rendimientos Acumulado "Equitativos"
anualizar_rendimiento(rendb_E,252)

1.55908720014391

In [ ]:
#Rendimientos Bruto "S&P500"
rendb_500 = calcular_rendimiento_bruto(rend["S&P500"].values[1], rend["S&P500"].iloc[-1], dias)
rendb_500

-0.42175180441092497

In [ ]:
#Rendimientos Acumulado "Equitativos"
anualizar_rendimiento(rendb_500,252)

-0.6025025777298928

In [ ]:
data = {
    'DATOS': ['RENDIMIENTO','SESGO','KURTOSIS','VaR','CVaR','SHARPE RATIO','SORTINO','DRAWDOWN'],
    'MIN_VAR': [rendb_MV,sesgo_general[0], kurtosis_general[0], VaR_95_general[0],CVaR_95_general[0],sharpe_ratio_general[0],sortino_ratio_general[0],drawdown_general[0]],
    'MÁX_SHARPE_RATIO': [rendb_MSR,sesgo_general[1], kurtosis_general[1], VaR_95_general[1],CVaR_95_general[1],sharpe_ratio_general[1],sortino_ratio_general[1],drawdown_general[1]],
    'MIN_VAR_10%': [rendb_MV10 ,sesgo_general[2], kurtosis_general[2], VaR_95_general[2],CVaR_95_general[2],sharpe_ratio_general[2],sortino_ratio_general[2],drawdown_general[2]],
    'S&P500': [rendb_500,sesgo_general[3], kurtosis_general[3], VaR_95_general[3],CVaR_95_general[3],sharpe_ratio_general[3],sortino_ratio_general[3],drawdown_general[3]]

}

### **Tabla General de los datos obtenidos**

In [ ]:
df_data = pd.DataFrame(data)
df_data

,DATOS,MIN_VAR,MÁX_SHARPE_RATIO,MIN_VAR_10%,S&P500
0,RENDIMIENTO,6.692746,0.389761,10.692732,-0.421752
1,SESGO,0.497599,0.090967,0.571096,0.085117
2,KURTOSIS,10.052346,1.134385,14.338017,2.689022
3,VaR,-0.013828,-0.029341,-0.015633,-0.016297
4,CVaR,-0.016656,-0.038399,-0.019388,-0.020834
5,SHARPE RATIO,-1.012815,0.220805,-0.751380,-0.431432
6,SORTINO,-6.966434,-3.036549,-6.177203,-5.495902
7,DRAWDOWN,-0.340972,-0.442657,-0.339172,-0.320986


Una vez obtenidas todas las metricas de riesgo en lso intervalos de tiempo dados y comparando dichos valores, para nuestro caso con los activos especificos: 'AGG', 'SEMB.L', 'IYW', 'IEMS.L', 'GSG'.

Nosotros decidimos tomar un portafolio menos expuesto al riesgo, por lo que nuestras opciones más convenientes son: el Portafolio Mínima volatilidad y el portafolio de Mínima Volatilidad con 10% de rendimientos.

Aquí estamos dejando a lado la posibilidad de estar arriesgados, por lo que nuestro riesgo-beneficio es más bajo que el que pudimos haber obtenido en el portafolio Mácimo Sharpe Ratio.

EL VaR y el CVaR nos indica que nuestros dos portafolios tiene valores esperados de perdida menor a los démas, pero en cambio la Kurtosis nos está indicando que nuestros potafolios cuentan con una mayor exposición a los cambios extremos. En este punto (VaR y CVaR) nuestros dos portafolios no difieren a más de 0.003, por ello esta diferencia tan minima con respecto a nuestro rendimiento nos termina decantando, en esta metrica por el portafolio de Minima Varianza y 10% de rendimientos.

A su vez observando los rendimientos anuales obtenidos por los portafolios, hay una visible mejoria en los rendimientos obtenidos por el portafolio de Minima Varianza con 10% de rendimiento, el cual es superior a los valores obtenidos por los demás portafolios y por el indice S&P500.

De igual manera la metrica Sortino aunque nos indica que el portafolio de Mínima Volatilidad con rendiemiento 10% presenta un mejor desempeñoen su valor que los demás portafolios.

Por lo que, decidimos elegir el de Mínima Volatilidad con 10% de rendimiento como el portafolio donde invertiriamos, pues de manera general responde mejor a las metricas de riesgo aplicadas y presenta un mejor rendimiento, dentro de los posibles parametros ya fijados, minimizando nuestra volatilidad.

### **SEGUNDA PARTE DEL PROYECTO**

In [ ]:
#Retornos diarios del 2010 al 2023
returns_1

,AGG,GSG,IEMS.L,IYW,SEMB.L
Date,,,,,
2010-01-04,0.000000,0.000000,0.000000,0.000000,0.000000
2010-01-05,-0.000211,0.005305,0.001856,-0.006271,-0.004592
2010-01-06,-0.006140,0.001660,0.006748,-0.012547,-0.003555
2010-01-07,0.000466,-0.003192,-0.002717,-0.002175,0.000341
2010-01-08,-0.005380,-0.003854,-0.001164,0.001615,-0.004376
...,...,...,...,...,...
2023-12-04,-0.014573,-0.018517,-0.016650,-0.023235,-0.004107
2023-12-05,0.022174,0.007945,0.012199,0.020943,0.019943
2023-12-06,-0.002928,-0.033087,0.000154,-0.014853,-0.003723


In [ ]:
#Matriz de covarianza de los retornos diarios

cov_matrix = returns_1.cov()
cov_matrix

,AGG,GSG,IEMS.L,IYW,SEMB.L
AGG,0.000072,0.000033,0.000034,0.000039,0.000026
GSG,0.000033,0.000193,0.000065,0.000071,0.000011
IEMS.L,0.000034,0.000065,0.000149,0.000095,0.000024
IYW,0.000039,0.000071,0.000095,0.000215,0.000031
SEMB.L,0.000026,0.000011,0.000024,0.000031,0.000112


In [ ]:
#Pesos equitativos
wts

array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])

In [ ]:
#Obteniendo la desviación estandar del mercado
sigma_m = np.dot(np.dot(wts.T,cov_matrix),wts)
sigma_m = sqrt(sigma_m)
sigma_m

array([[0.0079935]])

In [ ]:
#obtenemos lambda
lam = (1/sigma_m)*0.5
tau = (1/120)
lam

array([[62.55081543]])

In [ ]:
#Obengamos la distribución del prior
media_prior = lam * np.dot(cov_matrix,wts)
varianza_prior = tau * cov_matrix


In [ ]:
p= {
    'Relativo': [1, -1, 0, 0, 0], #Relativo
    'Absoluto': [0, 0, 0, 1, 0],#Absoluto
}

df_p = pd.DataFrame(p).transpose()
q = np.array([0.065,0.078])
df_p

,0,1,2,3,4
Relativo,1,-1,0,0,0
Absoluto,0,0,0,1,0


In [ ]:
#Vamos a obtener omega
a = np.dot(df_p,np.dot(tau * matrix_cov,df_p.T))
a[0][1] = 0
a[1][0] = 0

omega = a
omega



array([[1.51051219e-06, 0.00000000e+00],
       [0.00000000e+00, 1.53639519e-06]])

In [ ]:
#Obtengamos la distribución posteriori

factor1 = np.linalg.inv(np.linalg.inv(tau * cov_matrix) + np.dot(df_p.T,np.dot(np.linalg.inv(omega),df_p)))
factor2 = np.dot(media_prior.T,np.linalg.inv(tau * cov_matrix)) + np.dot(df_p.T, np.dot(np.linalg.inv(omega),q))

media_posteriori = np.dot(factor2, factor1)
media_posteriori = pd.DataFrame(media_posteriori)

varianza_posteriori = factor1
varianza_posteriori = pd.DataFrame(varianza_posteriori)



In [ ]:
l = [0.3,1,6,lam]
w = [0,0,0,0,0]
#(1/lam1)* np.dot(np.linalg.inv(cov_matrix),media_posteriori)
f1 = np.linalg.inv(cov_matrix)
for i in range(len(l)):
  w[i] = np.dot(l[i] * np.linalg.inv(cov_matrix),media_posteriori.T)

In [ ]:
#Lambda = 0.3, que corresponde a una inversión con muy bajo aversión al riesgo
print(w[0])
print(sum(w[0]))

[[ 61.67106168]
 [-54.16496383]
 [  3.75304893]
 [ 62.88056899]
 [  3.75304893]]
77.89276469057273


In [ ]:
#Lambda = 1, que corresponde al nivel más bajo en términos de riesgo
print(w[1])
print(sum(w[1]))

[[ 205.57020562]
 [-180.54987944]
 [  12.51016309]
 [ 209.60189662]
 [  12.51016309]]
259.64254896857574


In [ ]:
#lambda = 6, que corresponde a una inversión riesgosa
print(w[2])
print(sum(w[2]))

[[ 1233.4212337 ]
 [-1083.29927665]
 [   75.06097852]
 [ 1257.61137972]
 [   75.06097852]]
1557.855293811454


In [ ]:
#Lambda que calculamos con el prior
print(w[3])
print(sum(w[3]))

[[ 12858.58399041]
 [-11293.54218579]
 [   782.52090231]
 [ 13110.76955032]
 [   782.52090231]]
16240.853159582235


Notemos que la lambda que mejor nos acomoda es lambda = 0.3, que corresponde al riesgo más bajo en términos de adversión al riesgo, esto nos indica que el 22.11% se invertirá en la tasa libre de riesgo.